# Experiment 003: DINOv2-giant + Image Patching + 4-Model Ensemble

Following evolver strategy Priority 1:
1. Image patching (520px patches with overlap) to preserve spatial detail
2. DINOv2-giant (1536 dims) for more expressive features
3. 4-model ensemble (LightGBM, CatBoost, XGBoost, HistGradientBoosting)
4. Post-processing for biomass constraints

Target: CV ≥ 0.79

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from PIL import Image
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Verify GPU
print(f'CUDA available: {torch.cuda.is_available()}')
print(f'GPU: {torch.cuda.get_device_name(0)}')
print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

CUDA available: True
GPU: NVIDIA H100 80GB HBM3
Memory: 85.0 GB


In [2]:
# Load data
DATA_DIR = '/home/data'
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')

# Pivot train data
train_pivot = train_df.pivot_table(
    index=['image_path', 'Sampling_Date', 'State', 'Species', 'Pre_GSHH_NDVI', 'Height_Ave_cm'],
    columns='target_name',
    values='target'
).reset_index()

print(f'Pivoted train shape: {train_pivot.shape}')
print(f'Test shape: {test_df.shape}')

# Check image dimensions
sample_img = Image.open(f'{DATA_DIR}/{train_pivot["image_path"].iloc[0]}')
print(f'Sample image size: {sample_img.size}')

Pivoted train shape: (357, 11)
Test shape: (5, 3)
Sample image size: (2000, 1000)


In [3]:
# Load DINOv2-giant model (1536 dims)
from transformers import AutoImageProcessor, AutoModel

model_name = 'facebook/dinov2-giant'
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).cuda().eval()

print(f'Model loaded: {model_name}')
print(f'Hidden size: {model.config.hidden_size}')
print(f'Image size expected: {processor.size}')

2026-01-15 02:14:20.865117: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2026-01-15 02:14:20.880868: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2026-01-15 02:14:20.885324: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Model loaded: facebook/dinov2-giant
Hidden size: 1536
Image size expected: {'shortest_edge': 256}


In [4]:
# Image patching function - split image into 520px patches with overlap
def split_image_into_patches(image, patch_size=518, overlap=16):
    """Split image into overlapping patches.
    
    For 2000x1000 image with 518px patches and 16px overlap:
    - stride = 518 - 16 = 502
    - horizontal: ceil((2000-518)/502) + 1 = 4 patches
    - vertical: ceil((1000-518)/502) + 1 = 2 patches
    - Total: 8 patches per image
    """
    img_array = np.array(image)
    h, w, c = img_array.shape
    stride = patch_size - overlap
    
    patches = []
    for y in range(0, h, stride):
        for x in range(0, w, stride):
            # Extract patch
            y_end = min(y + patch_size, h)
            x_end = min(x + patch_size, w)
            patch = img_array[y:y_end, x:x_end]
            
            # Pad if needed
            if patch.shape[0] < patch_size or patch.shape[1] < patch_size:
                pad_h = patch_size - patch.shape[0]
                pad_w = patch_size - patch.shape[1]
                patch = np.pad(patch, ((0, pad_h), (0, pad_w), (0, 0)), mode='reflect')
            
            patches.append(Image.fromarray(patch))
    
    return patches

# Test patching
test_patches = split_image_into_patches(sample_img)
print(f'Number of patches per image: {len(test_patches)}')
print(f'Patch size: {test_patches[0].size}')

Number of patches per image: 8
Patch size: (518, 518)


In [5]:
# Extract patch-based embeddings with DINOv2-giant
def extract_patch_embeddings(image_paths, data_dir, batch_size=4):
    """Extract embeddings by averaging patch embeddings."""
    all_embeddings = []
    
    with torch.no_grad():
        for img_path in tqdm(image_paths):
            # Load image and split into patches
            img = Image.open(f'{data_dir}/{img_path}').convert('RGB')
            patches = split_image_into_patches(img)
            
            # Process patches in batches
            patch_embeddings = []
            for i in range(0, len(patches), batch_size):
                batch_patches = patches[i:i+batch_size]
                inputs = processor(images=batch_patches, return_tensors='pt').to('cuda')
                outputs = model(**inputs)
                
                # Use mean of patch tokens (excluding CLS)
                patch_tokens = outputs.last_hidden_state[:, 1:, :]  # (batch, num_patches, hidden)
                patch_mean = patch_tokens.mean(dim=1)  # (batch, hidden)
                patch_embeddings.append(patch_mean.cpu().numpy())
            
            # Average all patch embeddings for this image
            patch_embeddings = np.vstack(patch_embeddings)
            image_embedding = patch_embeddings.mean(axis=0)
            all_embeddings.append(image_embedding)
    
    return np.vstack(all_embeddings)

# Extract train embeddings
print('Extracting train embeddings with DINOv2-giant + patching...')
train_embeddings = extract_patch_embeddings(train_pivot['image_path'].values, DATA_DIR)
print(f'Train embeddings shape: {train_embeddings.shape}')

Extracting train embeddings with DINOv2-giant + patching...


  0%|          | 0/357 [00:00<?, ?it/s]

  0%|          | 1/357 [00:00<02:26,  2.42it/s]

  1%|          | 2/357 [00:00<01:52,  3.15it/s]

  1%|          | 3/357 [00:00<01:39,  3.55it/s]

...

 99%|█████████▉| 355/357 [01:25<00:00,  4.18it/s]

100%|█████████▉| 356/357 [01:26<00:00,  4.21it/s]

100%|██████████| 357/357 [01:26<00:00,  4.19it/s]

100%|██████████| 357/357 [01:26<00:00,  4.14it/s]

Train embeddings shape: (357, 1536)


In [6]:
# Extract test embeddings
print('Extracting test embeddings...')
test_images_unique = test_df['image_path'].unique()
test_embeddings = extract_patch_embeddings(test_images_unique, DATA_DIR)
print(f'Test embeddings shape: {test_embeddings.shape}')

Extracting test embeddings...


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  4.09it/s]

100%|██████████| 1/1 [00:00<00:00,  4.08it/s]

Test embeddings shape: (1, 1536)


In [7]:
# Clear GPU memory
import gc
del model
torch.cuda.empty_cache()
gc.collect()
print('GPU memory cleared')

GPU memory cleared


In [8]:
# Create feature dataframe
emb_cols = [f'emb_{i}' for i in range(train_embeddings.shape[1])]
train_emb_df = pd.DataFrame(train_embeddings, columns=emb_cols)
train_emb_df['image_path'] = train_pivot['image_path'].values

test_emb_df = pd.DataFrame(test_embeddings, columns=emb_cols)
test_emb_df['image_path'] = test_images_unique

print(f'Train embeddings df shape: {train_emb_df.shape}')
print(f'Test embeddings df shape: {test_emb_df.shape}')

Train embeddings df shape: (357, 1537)
Test embeddings df shape: (1, 1537)


In [9]:
# Prepare tabular features
from sklearn.preprocessing import LabelEncoder

le_state = LabelEncoder()
le_species = LabelEncoder()

all_states = pd.concat([train_pivot['State'], pd.Series(['Unknown'])])
all_species = pd.concat([train_pivot['Species'], pd.Series(['Unknown'])])

le_state.fit(all_states)
le_species.fit(all_species)

train_pivot['State_enc'] = le_state.transform(train_pivot['State'])
train_pivot['Species_enc'] = le_species.transform(train_pivot['Species'])

# Merge embeddings with tabular features
train_full = train_pivot.merge(train_emb_df, on='image_path')
print(f'Train full shape: {train_full.shape}')

# Define target columns and weights
target_cols = ['Dry_Green_g', 'Dry_Dead_g', 'Dry_Clover_g', 'GDM_g', 'Dry_Total_g']
target_weights = {'Dry_Green_g': 0.1, 'Dry_Dead_g': 0.1, 'Dry_Clover_g': 0.1, 'GDM_g': 0.2, 'Dry_Total_g': 0.5}

# Define feature columns (1536 DINOv2-giant + 4 tabular)
feature_cols = ['Pre_GSHH_NDVI', 'Height_Ave_cm', 'State_enc', 'Species_enc'] + emb_cols
print(f'Number of features: {len(feature_cols)}')

Train full shape: (357, 1549)
Number of features: 1540


In [10]:
# Define weighted R2 metric
def weighted_r2(y_true_dict, y_pred_dict, weights):
    all_y_true, all_y_pred, all_weights = [], [], []
    
    for target in y_true_dict.keys():
        all_y_true.extend(y_true_dict[target])
        all_y_pred.extend(y_pred_dict[target])
        all_weights.extend([weights[target]] * len(y_true_dict[target]))
    
    all_y_true = np.array(all_y_true)
    all_y_pred = np.array(all_y_pred)
    all_weights = np.array(all_weights)
    
    y_mean = np.sum(all_weights * all_y_true) / np.sum(all_weights)
    ss_res = np.sum(all_weights * (all_y_true - all_y_pred) ** 2)
    ss_tot = np.sum(all_weights * (all_y_true - y_mean) ** 2)
    
    return 1 - ss_res / ss_tot

# Post-processing function
def post_process_biomass(preds_dict):
    ordered_cols = ['Dry_Green_g', 'Dry_Clover_g', 'Dry_Dead_g', 'GDM_g', 'Dry_Total_g']
    Y = np.vstack([preds_dict[col] for col in ordered_cols])
    
    C = np.array([[1, 1, 0, -1, 0], [0, 0, 1, 1, -1]])
    C_T = C.T
    inv_CCt = np.linalg.inv(C @ C_T)
    P = np.eye(5) - C_T @ inv_CCt @ C
    
    Y_reconciled = (P @ Y).clip(min=0)
    
    return {col: Y_reconciled[i] for i, col in enumerate(ordered_cols)}

In [12]:
# 4-Model Ensemble with 5-Fold CV
import lightgbm as lgb
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import KFold

N_FOLDS = 5
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

# Store OOF predictions
oof_preds = {target: np.zeros(len(train_full)) for target in target_cols}
oof_preds_pp = {target: np.zeros(len(train_full)) for target in target_cols}
fold_scores = []
fold_scores_pp = []

X = train_full[feature_cols].values

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f'\n=== Fold {fold + 1} ===')
    
    X_train, X_val = X[train_idx], X[val_idx]
    
    fold_y_true = {}
    fold_y_pred = {}
    
    for target in target_cols:
        y = train_full[target].values
        y_train, y_val = y[train_idx], y[val_idx]
        
        # 4-Model Ensemble
        preds_list = []
        
        # 1. LightGBM
        lgb_model = lgb.LGBMRegressor(
            n_estimators=500, learning_rate=0.05, num_leaves=31,
            feature_fraction=0.8, bagging_fraction=0.8, bagging_freq=5,
            verbose=-1, random_state=42
        )
        lgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)],
                      callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)])
        preds_list.append(lgb_model.predict(X_val))
        
        # 2. CatBoost
        cb_model = CatBoostRegressor(
            iterations=500, learning_rate=0.05, depth=6,
            verbose=0, random_state=42
        )
        cb_model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)
        preds_list.append(cb_model.predict(X_val))
        
        # 3. XGBoost (fixed API)
        xgb_model = XGBRegressor(
            n_estimators=500, learning_rate=0.05, max_depth=6,
            subsample=0.8, colsample_bytree=0.8,
            early_stopping_rounds=50,
            verbosity=0, random_state=42
        )
        xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        preds_list.append(xgb_model.predict(X_val))
        
        # 4. HistGradientBoosting
        hgb_model = HistGradientBoostingRegressor(
            max_iter=500, learning_rate=0.05, max_depth=6,
            random_state=42
        )
        hgb_model.fit(X_train, y_train)
        preds_list.append(hgb_model.predict(X_val))
        
        # Average predictions
        ensemble_preds = np.mean(preds_list, axis=0)
        ensemble_preds = np.clip(ensemble_preds, 0, None)
        
        oof_preds[target][val_idx] = ensemble_preds
        fold_y_true[target] = y_val
        fold_y_pred[target] = ensemble_preds
    
    # Calculate fold R2
    fold_r2 = weighted_r2(fold_y_true, fold_y_pred, target_weights)
    fold_scores.append(fold_r2)
    
    # Apply post-processing
    fold_y_pred_pp = post_process_biomass(fold_y_pred)
    fold_r2_pp = weighted_r2(fold_y_true, fold_y_pred_pp, target_weights)
    fold_scores_pp.append(fold_r2_pp)
    
    for target in target_cols:
        oof_preds_pp[target][val_idx] = fold_y_pred_pp[target]
    
    print(f'Fold {fold + 1} Weighted R2: {fold_r2:.4f} -> {fold_r2_pp:.4f} (post-processed)')

print(f'\n=== Overall CV Results ===')
print(f'Mean Weighted R2 (raw): {np.mean(fold_scores):.4f} (+/- {np.std(fold_scores):.4f})')
print(f'Mean Weighted R2 (post-processed): {np.mean(fold_scores_pp):.4f} (+/- {np.std(fold_scores_pp):.4f})')


=== Fold 1 ===
Training until validation scores don't improve for 50 rounds


Early stopping, best iteration is:
[226]	valid_0's l2: 84.3145


Training until validation scores don't improve for 50 rounds


Early stopping, best iteration is:
[234]	valid_0's l2: 67.3999


Training until validation scores don't improve for 50 rounds


...

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[125]	valid_0's l2: 71.2964


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[55]	valid_0's l2: 44.8635


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[118]	valid_0's l2: 192.658


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[114]	valid_0's l2: 279.859


Fold 5 Weighted R2: 0.7438 -> 0.7431 (post-processed)

=== Overall CV Results ===
Mean Weighted R2 (raw): 0.8138 (+/- 0.0355)
Mean Weighted R2 (post-processed): 0.8185 (+/- 0.0384)


In [13]:
# Calculate overall OOF weighted R2
oof_y_true = {target: train_full[target].values for target in target_cols}
overall_r2 = weighted_r2(oof_y_true, oof_preds, target_weights)
overall_r2_pp = weighted_r2(oof_y_true, oof_preds_pp, target_weights)

print(f'Overall OOF Weighted R2 (raw): {overall_r2:.4f}')
print(f'Overall OOF Weighted R2 (post-processed): {overall_r2_pp:.4f}')

Overall OOF Weighted R2 (raw): 0.8148
Overall OOF Weighted R2 (post-processed): 0.8197


In [14]:
# Train final ensemble models on full data
print('Training final ensemble models on full data...')
final_models = {target: [] for target in target_cols}
X_full = train_full[feature_cols].values

for target in target_cols:
    print(f'Training models for {target}...')
    y_full = train_full[target].values
    
    # LightGBM
    lgb_model = lgb.LGBMRegressor(
        n_estimators=500, learning_rate=0.05, num_leaves=31,
        feature_fraction=0.8, bagging_fraction=0.8, bagging_freq=5,
        verbose=-1, random_state=42
    )
    lgb_model.fit(X_full, y_full)
    final_models[target].append(lgb_model)
    
    # CatBoost
    cb_model = CatBoostRegressor(
        iterations=500, learning_rate=0.05, depth=6,
        verbose=0, random_state=42
    )
    cb_model.fit(X_full, y_full)
    final_models[target].append(cb_model)
    
    # XGBoost
    xgb_model = XGBRegressor(
        n_estimators=500, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8,
        verbosity=0, random_state=42
    )
    xgb_model.fit(X_full, y_full)
    final_models[target].append(xgb_model)
    
    # HistGradientBoosting
    hgb_model = HistGradientBoostingRegressor(
        max_iter=500, learning_rate=0.05, max_depth=6,
        random_state=42
    )
    hgb_model.fit(X_full, y_full)
    final_models[target].append(hgb_model)

print('All final models trained!')

Training final ensemble models on full data...
Training models for Dry_Green_g...


Training models for Dry_Dead_g...


Training models for Dry_Clover_g...


Training models for GDM_g...


Training models for Dry_Total_g...


All final models trained!


In [15]:
# Prepare test features
test_features = test_emb_df.copy()
test_features['Pre_GSHH_NDVI'] = train_pivot['Pre_GSHH_NDVI'].mean()
test_features['Height_Ave_cm'] = train_pivot['Height_Ave_cm'].mean()
test_features['State_enc'] = train_pivot['State_enc'].mode()[0]
test_features['Species_enc'] = train_pivot['Species_enc'].mode()[0]

X_test = test_features[feature_cols].values
print(f'Test features shape: {X_test.shape}')

Test features shape: (1, 1540)


In [16]:
# Make ensemble predictions for test set
test_preds = {}
for target in target_cols:
    preds_list = [model.predict(X_test) for model in final_models[target]]
    ensemble_preds = np.mean(preds_list, axis=0)
    ensemble_preds = np.clip(ensemble_preds, 0, None)
    test_preds[target] = ensemble_preds
    print(f'{target}: mean={ensemble_preds.mean():.2f}')

# Apply post-processing
test_preds_pp = post_process_biomass(test_preds)
print('\nAfter post-processing:')
for target in target_cols:
    print(f'{target}: mean={test_preds_pp[target].mean():.2f}')

Dry_Green_g: mean=30.42
Dry_Dead_g: mean=32.61
Dry_Clover_g: mean=0.18
GDM_g: mean=35.29
Dry_Total_g: mean=68.84

After post-processing:
Dry_Green_g: mean=32.30
Dry_Dead_g: mean=33.55
Dry_Clover_g: mean=2.06
GDM_g: mean=34.35
Dry_Total_g: mean=67.90


In [17]:
# Create submission file
submission_rows = []

for i, img_path in enumerate(test_images_unique):
    img_id = img_path.split('/')[-1].replace('.jpg', '')
    
    for target in target_cols:
        sample_id = f'{img_id}__{target}'
        pred_value = test_preds_pp[target][i]
        submission_rows.append({'sample_id': sample_id, 'target': pred_value})

submission_df = pd.DataFrame(submission_rows)
print(f'Submission shape: {submission_df.shape}')
print(submission_df)

Submission shape: (5, 2)
                    sample_id     target
0   ID1001187975__Dry_Green_g  32.297326
1    ID1001187975__Dry_Dead_g  33.550007
2  ID1001187975__Dry_Clover_g   2.057563
3         ID1001187975__GDM_g  34.354888
4   ID1001187975__Dry_Total_g  67.904895


In [18]:
# Save submission
submission_df.to_csv('/home/submission/submission.csv', index=False)
print('Submission saved to /home/submission/submission.csv')

Submission saved to /home/submission/submission.csv


In [19]:
# Final summary
print('='*70)
print('EXPERIMENT 003 RESULTS SUMMARY')
print('='*70)
print(f'Model: DINOv2-giant + Image Patching + 4-Model Ensemble + Post-processing')
print(f'Features: {len(feature_cols)} (1536 DINOv2-giant patch + 4 tabular)')
print(f'Ensemble: LightGBM, CatBoost, XGBoost, HistGradientBoosting')
print(f'CV Folds: {N_FOLDS}')
print(f'\nRaw predictions:')
print(f'  Mean CV Weighted R2: {np.mean(fold_scores):.4f} (+/- {np.std(fold_scores):.4f})')
print(f'  Overall OOF Weighted R2: {overall_r2:.4f}')
print(f'\nPost-processed predictions:')
print(f'  Mean CV Weighted R2: {np.mean(fold_scores_pp):.4f} (+/- {np.std(fold_scores_pp):.4f})')
print(f'  Overall OOF Weighted R2: {overall_r2_pp:.4f}')
print(f'\nComparison:')
print(f'  Baseline (exp_000): 0.7584')
print(f'  DINOv2-large patch (exp_001): 0.7715')
print(f'  This experiment: {overall_r2_pp:.4f} ({overall_r2_pp - 0.7715:+.4f} vs exp_001)')
print(f'\nTarget: 0.79, Gap: {0.79 - overall_r2_pp:.4f}')
print('='*70)

EXPERIMENT 003 RESULTS SUMMARY
Model: DINOv2-giant + Image Patching + 4-Model Ensemble + Post-processing
Features: 1540 (1536 DINOv2-giant patch + 4 tabular)
Ensemble: LightGBM, CatBoost, XGBoost, HistGradientBoosting
CV Folds: 5

Raw predictions:
  Mean CV Weighted R2: 0.8138 (+/- 0.0355)
  Overall OOF Weighted R2: 0.8148

Post-processed predictions:
  Mean CV Weighted R2: 0.8185 (+/- 0.0384)
  Overall OOF Weighted R2: 0.8197

Comparison:
  Baseline (exp_000): 0.7584
  DINOv2-large patch (exp_001): 0.7715
  This experiment: 0.8197 (+0.0482 vs exp_001)

Target: 0.79, Gap: -0.0297
